In [1]:
import torch
import torch.nn as nn
import torch.optim as optim


In [2]:
class PoissonRegression(nn.Module):
    def __init__(self, input_dim):
        super(PoissonRegression, self).__init__()
        self.linear = nn.Linear(input_dim, 1)

    def forward(self, x):
        return torch.exp(self.linear(x))


generate data

In [3]:
import numpy as np
import random

# Generate the artificial data

n_total = int(20000)
K = 100
X = np.column_stack((np.ones(n_total), np.random.rand(n_total * (K - 1)).reshape(n_total, K - 1)))
b0 = 1.0 * np.ones(K) / K
y = np.random.poisson(np.exp(X.dot(b0)))


In [4]:
# Split the data into train and test sets
n = len(y)
test_indices = random.sample(range(n), round(0.2 * n))

y_test = y[test_indices]
X_test = X[test_indices, :]

y_train = np.delete(y, test_indices)
X_train = np.delete(X, test_indices, axis=0)

# convert X_train into a pytorch tensor
X_train = torch.from_numpy(X_train).float()
y_train = torch.from_numpy(y_train).float()

In [5]:
model = PoissonRegression(K)
optimizer = optim.SGD(model.parameters(), lr=1)

In [6]:
def poisson_nll_loss(y_pred, y_true):
    return torch.mean(y_pred - y_true * torch.log(y_pred)) # this is the correct formula. 
# y_pred is the linear index, y_true is the true value.



In [7]:
optimizer = optim.SGD(model.parameters(), lr=0.01)

for epoch in range(100):  # number of epochs
    optimizer.zero_grad()
    y_pred = model(X_train) # the liner index
    loss = poisson_nll_loss(y_pred, y_train)
    loss.backward()
    optimizer.step()

    if epoch % 1 == 0:  # print every 100 epochs
        print(f'Epoch {epoch+1}, Loss: {loss.item()}')


Epoch 1, Loss: 1.1003668308258057
Epoch 2, Loss: 0.9636091589927673
Epoch 3, Loss: 0.8922603726387024
Epoch 4, Loss: 0.8599673509597778
Epoch 5, Loss: 0.8470344543457031
Epoch 6, Loss: 0.8423025012016296
Epoch 7, Loss: 0.8406559228897095
Epoch 8, Loss: 0.8400800228118896
Epoch 9, Loss: 0.8398569822311401
Epoch 10, Loss: 0.8397468328475952
Epoch 11, Loss: 0.8396721482276917
Epoch 12, Loss: 0.8396086096763611
Epoch 13, Loss: 0.839548647403717
Epoch 14, Loss: 0.8394898772239685
Epoch 15, Loss: 0.839431643486023
Epoch 16, Loss: 0.8393736481666565
Epoch 17, Loss: 0.8393158316612244
Epoch 18, Loss: 0.8392581939697266
Epoch 19, Loss: 0.8392006754875183
Epoch 20, Loss: 0.8391433358192444
Epoch 21, Loss: 0.8390861749649048
Epoch 22, Loss: 0.8390291929244995
Epoch 23, Loss: 0.8389724493026733
Epoch 24, Loss: 0.8389157056808472
Epoch 25, Loss: 0.8388592004776001
Epoch 26, Loss: 0.8388028144836426
Epoch 27, Loss: 0.8387466073036194
Epoch 28, Loss: 0.8386905193328857
Epoch 29, Loss: 0.8386346697807

In [8]:
# print the coefficients of the model

# print(model.state_dict()['linear.weight'])
print(model.state_dict())

OrderedDict([('linear.weight', tensor([[ 0.1144,  0.0591,  0.0363, -0.0234, -0.0331,  0.0234,  0.0042,  0.0778,
         -0.0607,  0.0329,  0.0043,  0.0930, -0.0373,  0.0163, -0.0069, -0.0654,
         -0.0276, -0.0438,  0.0172,  0.0405, -0.0676, -0.0195, -0.0185,  0.0292,
          0.0843, -0.0662, -0.0750,  0.0204,  0.0678,  0.0853,  0.0416, -0.0180,
          0.0500, -0.0008,  0.0454, -0.0146, -0.0568,  0.0249,  0.0283,  0.0454,
          0.0639, -0.0489, -0.0442,  0.0788,  0.0266,  0.0544,  0.0533, -0.0238,
          0.0917, -0.0546, -0.0019, -0.0618, -0.0693,  0.0072, -0.0689, -0.0681,
         -0.0014, -0.0111, -0.0213,  0.0823, -0.0713,  0.0230,  0.0167, -0.0067,
         -0.0458, -0.0031,  0.0464,  0.0140,  0.0736,  0.0423, -0.0470, -0.0199,
          0.0314, -0.0045, -0.0694,  0.0891,  0.0715, -0.0007,  0.0555,  0.0116,
         -0.0011,  0.0347, -0.0020, -0.0372, -0.0108, -0.0502,  0.0958, -0.0288,
          0.0827,  0.0138, -0.0731,  0.0508,  0.0811,  0.0830,  0.0303, -0.040